In [1]:
import torch
import os
import numpy

from torch_geometric.data import Dataset, Data
from comm_utils import generate_wGaussian, np_sum_rate_all


class d2dGraphDataset(Dataset):
    def __init__(self, num_samples, num_D2D, p_max, n0, seed=1309):
        self.num_samples = num_samples
        self.num_D2D = num_D2D
        self.p_max = p_max  # Transmit power
        self.n0 = n0  # Noise power
        self.seed = seed

        # Generate the dataset (graphs and labels)
        self.channels, self.w_sumrate, self.objs, self.labels = self.generate_data()
        self.graphs = [self.create_graph(idx) for idx in range(self.num_samples)]


    def generate_data(self):
        channel_matrices, power_matrices, weight_matrices, _ = generate_wGaussian(
            self.num_D2D, self.num_samples, seed=self.seed, var_noise = self.n0
        )

        wmmse_sumrate = np_sum_rate_all(channel_matrices,power_matrices,weight_matrices,self.n0)[:,None]

        print(f'Benchmark WMMSE: {numpy.mean(wmmse_sumrate)}')
        return channel_matrices.transpose(0,2,1), weight_matrices, wmmse_sumrate, power_matrices

    def create_graph(self, idx):
        H = self.channels[idx] 
        W = self.w_sumrate[idx] #.repeat(self.num_D2D, 1)
        x1 = torch.tensor(H.diagonal()[:, None], dtype=torch.float32)
        x2 = torch.tensor(W[:, None], dtype=torch.float32)
        x = torch.cat([x1, x2], dim=1)  # shape: [num_D2D, 2]
        # Fully connected graph
        edge_index = torch.tensor([
            # [i, j] for i in range(self.num_D2D) for j in range(self.num_D2D) if i != j
            [i, j] for i in range(self.num_D2D) for j in range(i+1, self.num_D2D)
        ], dtype=torch.long).T  # shape: [2, num_edges]

        H = torch.tensor(H, dtype=torch.float32)
        # edge_attr = torch.tensor(H[edge_index[0], edge_index[1]][:, None], dtype=torch.float32)
        edge_attr = torch.cat([H[edge_index[0], edge_index[1]][:, None], H[edge_index[1], edge_index[0]][:, None]], dim=-1)
        #  torch.tensor(H[edge_index[0], edge_index[1]][:, None], dtype=torch.float32)

        y = torch.tensor(self.labels[idx], dtype=torch.float32)

        data = Data(
            x=x,
            edge_index=edge_index,
            edge_attr=edge_attr,
            y=y
        )
        return data

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.graphs[idx]



In [6]:
def get_directLink_channel_losses(channel_losses):
    return np.diagonal(channel_losses, axis1=1, axis2=2)  # layouts X N

def get_crossLink_channel_losses(channel_losses):
    N = np.shape(channel_losses)[-1]
    return channel_losses * ((np.identity(N) < 1).astype(float))

def FP(weights, g, var_noise, input_x):
    number_of_samples, N, _ = np.shape(g)
    assert np.shape(g)==(number_of_samples, N, N)
    assert np.shape(weights)==(number_of_samples, N)
    g_diag = get_directLink_channel_losses(g)
    g_nondiag = get_crossLink_channel_losses(g)
    # For matrix multiplication and dimension matching requirement, reshape into column vectors
    weights = np.expand_dims(weights, axis=-1)
    g_diag = np.expand_dims(g_diag, axis=-1)
    x = input_x#np.ones([number_of_samples, N, 1])
    #tx_power = general_para.tx_power
    #output_noise_power = general_para.output_noise_power
    # tx_powers = np.ones([number_of_samples, N, 1]) * tx_power  # assume same power for each transmitter
    # Run 100 iterations of FP computations
    # In the computation below, every step's output is with shape: number of samples X N X 1
    for i in range(100):
        # Compute z
        p_x_prod = x 
        z_denominator = np.matmul(g_nondiag, p_x_prod) + var_noise
        z_numerator = g_diag * p_x_prod
        z = z_numerator / z_denominator
        # compute y
        y_denominator = np.matmul(g, p_x_prod) + var_noise
        y_numerator = np.sqrt(z_numerator * weights * (z + 1))
        y = y_numerator / y_denominator
        # compute x
        x_denominator = np.matmul(np.transpose(g, (0,2,1)), np.power(y, 2))
        x_numerator = y * np.sqrt(weights * (z + 1) * g_diag)
        x_new = np.power(x_numerator / x_denominator, 2)
        x_new[x_new > 1] = 1  # thresholding at upperbound 1
        x = x_new
    assert np.shape(x)==(number_of_samples, N, 1)
    x_final = np.squeeze(x, axis=-1)
    return x_final

In [7]:
noise = 10
train_size = 100
test_size = 10
num_nodes = 10
power = 1
seed = 1712

var_noise = 1/10**(noise/10)  # Convert dB to linear scale  
train_dataset = d2dGraphDataset(num_samples=train_size, num_D2D=num_nodes, p_max=power, n0=var_noise, seed=seed)
test_dataset = d2dGraphDataset(num_samples=test_size, num_D2D=num_nodes, p_max=power, n0=var_noise, seed=seed)

Generate Data ... (seed = 1712)
Benchmark WMMSE: 2.6138541945601905
Generate Data ... (seed = 1712)
Benchmark WMMSE: 2.438240816818398


In [11]:
def np_sum_rate(H,p,alpha,var_noise):
    H = numpy.expand_dims(H,axis=-1)
    K = H.shape[1]
    N = H.shape[-1]
    p = p.reshape((-1,K,1,N))
    rx_power = numpy.multiply(H, p)
    rx_power = numpy.sum(rx_power,axis=-1)
    rx_power = numpy.square(abs(rx_power))
    mask = numpy.eye(K)
    valid_rx_power = numpy.sum(numpy.multiply(rx_power, mask), axis=1)
    interference = numpy.sum(numpy.multiply(rx_power, 1 - mask), axis=1) + var_noise
    rate = numpy.log(1 + numpy.divide(valid_rx_power, interference))
    w_rate = numpy.multiply(alpha,rate)
    sum_rate = numpy.mean(numpy.sum(w_rate, axis=1))
    return sum_rate


In [26]:
import numpy as np

# Step 1: Generate data
channel_matrices, power_matrices, weight_matrices, _ = generate_wGaussian(K=num_nodes, num_H=train_size, var_noise=var_noise)

# Step 2: Initial random power
init_x = np.random.rand(train_size, num_nodes, 1)

# Step 3: Run FP
FP_powers = FP(weights=weight_matrices, g=channel_matrices, var_noise=var_noise, input_x=init_x)


wmmse_sumrate = np_sum_rate_all(channel_matrices,power_matrices,weight_matrices,var_noise)[:,None]
rates_fp = np_sum_rate_all(channel_matrices,  FP_powers, weight_matrices, var_noise)[:,None]

Generate Data ... (seed = 2017)


In [32]:
wmmse_sumrate

array([[2.10828193],
       [1.65682219],
       [1.76466185],
       [1.99581956],
       [2.41339518],
       [2.95564571],
       [3.55285501],
       [2.62993858],
       [1.53960564],
       [2.36373975],
       [1.6712038 ],
       [3.17375909],
       [2.79460227],
       [1.85104666],
       [2.38424145],
       [2.2636274 ],
       [2.24677789],
       [1.7515293 ],
       [0.74144779],
       [1.99198364],
       [1.46236759],
       [2.07890913],
       [2.31344546],
       [2.80648664],
       [2.47101451],
       [2.33481777],
       [1.71355256],
       [2.57320089],
       [2.17775646],
       [1.30222894],
       [3.45009408],
       [2.57529425],
       [2.24310529],
       [2.77861192],
       [3.01535998],
       [2.55095281],
       [3.00813441],
       [2.48714119],
       [1.56457481],
       [1.97744958],
       [2.31217326],
       [2.52687888],
       [1.93344304],
       [2.38796886],
       [3.01142743],
       [3.55909636],
       [1.727072  ],
       [2.148

In [34]:
rates_fp

array([[2.10828193],
       [1.57083562],
       [1.93798645],
       [1.53232442],
       [3.22455843],
       [2.95564571],
       [3.55285501],
       [2.62993858],
       [1.5396012 ],
       [1.39519027],
       [1.71965064],
       [4.1905537 ],
       [4.22870476],
       [2.14113117],
       [2.38424145],
       [2.45201013],
       [2.46194373],
       [2.46207487],
       [2.42349246],
       [1.87013865],
       [2.38466288],
       [2.08087202],
       [2.30945474],
       [2.80648664],
       [2.45224964],
       [2.3307757 ],
       [1.71355256],
       [2.57320089],
       [2.25745531],
       [1.30222894],
       [2.60024896],
       [2.88467247],
       [2.61495427],
       [2.77861192],
       [3.01535998],
       [3.46725699],
       [3.00813441],
       [2.0264256 ],
       [1.54684338],
       [1.97744958],
       [2.31217326],
       [2.75611078],
       [1.8841855 ],
       [1.79022398],
       [3.01142743],
       [3.55909636],
       [2.07981023],
       [1.564

In [1]:
from data import d2dGraphDataset
from torch_geometric.loader import DataLoader

In [3]:
var_noise = 1/10**(10/10)  # Convert dB to linear scale  
train_dataset = d2dGraphDataset(num_samples=2, num_D2D=10, p_max=1, n0=var_noise, seed=12)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


Generate Data ... (seed = 12)
Benchmark WMMSE: 2.6256741139015545
Benchmark FP: 2.591225203749303


In [5]:
import numpy

total_wmmse = 0
total_fp = 0
num_batch = 0
    
for data in train_loader:
    total_wmmse += numpy.sum(data.y[:,0].detach().numpy())
    total_fp += numpy.sum(data.y[:,1].detach().numpy())
    num_batch += data.num_graphs
train_wmmse = total_wmmse/num_batch
train_fp = total_fp/num_batch

In [12]:
data.y[:,0].detach().numpy()

array([1.3188488, 2.7430751], dtype=float32)

In [7]:
data.x

tensor([[0.9553, 0.1542],
        [0.7724, 0.7400],
        [0.3812, 0.2633],
        [0.8641, 0.5337],
        [0.5361, 0.0146],
        [0.7428, 0.9187],
        [0.6098, 0.9007],
        [0.5246, 0.0334],
        [1.0836, 0.9569],
        [0.7628, 0.1372],
        [0.9723, 0.2838],
        [1.2023, 0.6061],
        [0.2659, 0.9442],
        [1.5514, 0.8527],
        [1.0507, 0.0023],
        [0.6741, 0.5212],
        [1.1273, 0.5520],
        [0.6372, 0.4854],
        [0.4172, 0.7681],
        [1.1358, 0.1607]])